In [10]:
print('-------------------------------------from ct2000 to ct2010---------------------------------------------------------------')

-------------------------------------from ct2000 to ct2010---------------------------------------------------------------


In [1]:
import pandas as pd
import numpy as np

In [195]:
ct2010 = pd.read_csv("2010.csv")
ct2000 = pd.read_csv("2000.csv")

In [196]:
ct2000.head()

,CTLabel,BoroCode,BoroName,CT2000,BoroCT2000,CDEligibil,NTACode,NTANAme,PUMA,Shape_Leng,Shape_Area
0,32.0,1,Manhattan,3200,1003200,E,MN22,East Village,3809,6358.386615,2.334190e+06
1,33.0,1,Manhattan,3300,1003300,I,MN24,SoHo-TriBeCa-Civic Center-Little Italy,3810,8170.738948,3.677832e+06
2,34.0,1,Manhattan,3400,1003400,E,MN22,East Village,3809,5503.258021,1.717139e+06
3,38.0,1,Manhattan,3800,1003800,I,MN22,East Village,3809,5841.732079,2.180725e+06
4,39.0,1,Manhattan,3900,1003900,I,MN24,SoHo-TriBeCa-Civic Center-Little Italy,3810,8478.262635,3.000514e+06


In [197]:
ct2010.head()

,borocode,boroct2010,boroname,cdeligibil,ct2010,ctlabel,ntacode,ntaname,puma,shape_area,shape_leng
0,5,5000900,Staten Island,I,900,9.00,SI22,West New Brighton-New Brighton-St. George,3903,2.497010e+06,7729.016729
1,5,5012500,Staten Island,I,12500,125.00,SI22,West New Brighton-New Brighton-St. George,3903,4.954936e+06,10582.855530
2,5,5013301,Staten Island,E,13301,133.01,SI22,West New Brighton-New Brighton-St. George,3903,7.217847e+05,3428.312676
3,5,5019700,Staten Island,I,19700,197.00,SI07,Westerleigh,3903,3.231720e+06,9097.697231
4,5,5002002,Staten Island,I,2002,20.02,SI14,Grasmere-Arrochar-Ft. Wadsworth,3902,8.501224e+06,12591.725901


In [3]:
print(ct2010.dtypes)
print(ct2000.dtypes)

borocode        int64
boroct2010      int64
boroname       object
cdeligibil     object
ct2010          int64
ctlabel       float64
ntacode        object
ntaname        object
puma            int64
shape_area    float64
shape_leng    float64
dtype: object
CTLabel       float64
BoroCode        int64
BoroName       object
CT2000          int64
BoroCT2000      int64
CDEligibil     object
NTACode        object
NTANAme        object
PUMA            int64
Shape_Leng    float64
Shape_Area    float64
dtype: object


In [4]:
boroct2000 = set(ct2000.BoroCT2000)
boroct2010 = set(ct2010.boroct2010)

common = boroct2010 & boroct2000
unique_2010 = boroct2010 - common
unique_2000 = boroct2000 - common

print('number of ct in both ct2000 and ct2010:')
print(len(common))
print('number of ct only in ct2010')
print(len(unique_2010))
print('number of ct only in ct2000')
print(len(unique_2000))


In [ ]:
import shapefile

sp2000 = shapefile.Reader("nyct2000/nyct2000")

sp2010 = shapefile.Reader("nyct2010/nyct2010")

print(sp2000.fields)
print(sp2010.fields)

def posDict(sp, field):
    result = {}
    for rec in sp.records():
        result[int(rec[field])] = rec.oid
    return result

sp2000_boroct2000_posDict = posDict(sp2000, 'BoroCT2000')
sp2010_boroct2010_posDict = posDict(sp2010, 'boroct2010')

In [ ]:
from shapely.geometry import Polygon

def seperatePoints(points, breakings):
    result = []
    for i in range(len(breakings)):
        breaking = breakings[i]
        try:
            next_breaking = breakings[i+1]
        except:
            next_breaking = len(points)
        result.append(points[breaking:next_breaking])
    return result
            

def getPolygons(shape):
    polygons = []
    points = seperatePoints(shape.points, shape.parts)
    for polygon in points:
        polygons.append(Polygon(polygon))
    return polygons

def findIntersections(polygons1,polygons2):
    intersections = Polygon()
    for polygon1 in polygons1:
        for polygon2 in polygons2:
            intersections = intersections.union(polygon1.intersection(polygon2))
    return intersections

"""
def findIntersections(shape1,shape2):
    polygons1 = getPolygons(shape1)
    polygons2 = getPolygons(shape2)
    intersections = Polygon()
    for polygon1 in polygons1:
        for polygon2 in polygons2:
            intersections = intersections.union(polygon1.intersection(polygon2))
    return intersections
""" and None


In [137]:

# boroPolygons_pool: {'1':[{'label':ct2000_1,'polygons':polygons_1,'area':polygons_1_area},{...},...],'2':[...],...}
boroPolygons_pool = {'1':[],'2':[],'3':[],'4':[],'5':[]}
for this_ct2010 in unique_2010:
    this_shape = sp2010.shape(sp2010_boroct2010_posDict[this_ct2010])
    this_polygons = getPolygons(this_shape)
    this_dict = {}
    this_dict['label'] = this_ct2010
    this_dict['polygons'] = this_polygons.copy()
    this_dict['area'] = Polygon(this_shape.points).area
    boroPolygons_pool[str(this_ct2010)[0]].append(this_dict.copy())


{'1': [{1031703: [<shapely.geometry.polygon.Polygon at 0x1ed0450c860>]},
  {1031704: [<shapely.geometry.polygon.Polygon at 0x1ed0450ca20>]},
  {1021500: [<shapely.geometry.polygon.Polygon at 0x1ed044f5978>]},
  {1023100: [<shapely.geometry.polygon.Polygon at 0x1ed044a8d30>]},
  {1025700: [<shapely.geometry.polygon.Polygon at 0x1ed044e96a0>]},
  {1003700: [<shapely.geometry.polygon.Polygon at 0x1ed0450ce80>]},
  {1024200: [<shapely.geometry.polygon.Polygon at 0x1ed045132e8>]},
  {1022700: [<shapely.geometry.polygon.Polygon at 0x1ed04513be0>]},
  {1021703: [<shapely.geometry.polygon.Polygon at 0x1ed04513dd8>]},
  {1029900: [<shapely.geometry.polygon.Polygon at 0x1ed04513d30>,
    <shapely.geometry.polygon.Polygon at 0x1ed04513f28>]},
  {1023801: [<shapely.geometry.polygon.Polygon at 0x1ed044b7668>]},
  {1023802: [<shapely.geometry.polygon.Polygon at 0x1ed044a09b0>]},
  {1025900: [<shapely.geometry.polygon.Polygon at 0x1ed044b7748>]},
  {1017200: [<shapely.geometry.polygon.Polygon at 0x1e

In [6]:
# for the following dictionary, the key is boroct2000 and value is the corresponding boroct2010
map_2000_to_2010 = {}

for this_ct2000 in unique_2000:
    potential_pool = boroPolygons_pool[str(this_ct2000)[0]]
    max_overlap = 0
    shape00 = sp2000.shape(sp2000_boroct2000_posDict[this_ct2000])
    polygons00 = getPolygons(shape00)
    area00 = 0
    for temp in polygons00:
        area00 = area00 + temp.area
    # for the following dict: {'label':ct2010,'polygons':polygons,'area':area}
    for dict_10 in potential_pool:
        this_intersections = findIntersections(polygons00,dict_10['polygons'])
        intersections_area = this_intersections.area
        this_overlap = max(intersections_area / area00, intersections_area / dict_10['area'])
        if this_overlap > 0.1 and this_overlap > max_overlap:
            max_overlap = this_overlap
            corresponding = dict_10['label']
    if max_overlap > 0:
        map_2000_to_2010[this_ct2000] = corresponding


,borocode,boroct2010,boroname,cdeligibil,ct2010,ctlabel,ntacode,ntaname,puma,shape_area,shape_leng
1288,3,3072000,Brooklyn,I,72000,720.0,BK58,Flatlands,4009,1.701872e+06,6664.51608


In [230]:
for this_ct2000 in common:
    map_2000_to_2010[this_ct2000] = this_ct2000
special_ct2000 = unique_2000 - set(map_2000_to_2010.keys())

---------------finished--------------------------


In [8]:
print('-------------------------------------------------------finished----------------------------------------------------------')

-------------------------------------------------------finished----------------------------------------------------------


In [7]:
import shapefile

sp2000 = shapefile.Reader("nyct2000/nyct2000")

sp2010 = shapefile.Reader("nyct2010/nyct2010")

print(sp2000.fields)
print(sp2010.fields)

[('DeletionFlag', 'C', 1, 0), ['CTLabel', 'C', 7, 0], ['BoroCode', 'C', 1, 0], ['BoroName', 'C', 32, 0], ['CT2000', 'C', 6, 0], ['BoroCT2000', 'C', 7, 0], ['CDEligibil', 'C', 1, 0], ['NTACode', 'C', 4, 0], ['NTANAme', 'C', 75, 0], ['PUMA', 'C', 4, 0], ['Shape_Leng', 'F', 19, 11], ['Shape_Area', 'F', 19, 11]]
[('DeletionFlag', 'C', 1, 0), ['borocode', 'C', 254, 0], ['boroct2010', 'C', 254, 0], ['boroname', 'C', 254, 0], ['cdeligibil', 'C', 254, 0], ['ct2010', 'C', 254, 0], ['ctlabel', 'C', 254, 0], ['ntacode', 'C', 254, 0], ['ntaname', 'C', 254, 0], ['puma', 'C', 254, 0], ['shape_area', 'N', 33, 31], ['shape_leng', 'N', 33, 31]]


In [22]:
test = 3070203
test2000 = sp2000.shape(sp2000_boroct2000_posDict[test])
test2010 = sp2010.shape(sp2010_boroct2010_posDict[test])

In [113]:

def seperatePoints(points, breakings):
    result = []
    for i in range(len(breakings)):
        breaking = breakings[i]
        try:
            next_breaking = breakings[i+1]
        except:
            next_breaking = len(points)
        result.append(points[breaking:next_breaking])
    return result
            

def getPolygons(shape):
    polygons = []
    points = seperatePoints(shape.points, shape.parts)
    for polygon in points:
        polygons.append(Polygon(polygon))
    return polygons

def findIntersections(polygons1,polygons2):
    intersections = Polygon()
    for polygon1 in polygons1:
        for polygon2 in polygons2:
            intersections = intersections.union(polygon1.intersection(polygon2))
    return intersections

"""
def findIntersections(shape1,shape2):
    polygons1 = getPolygons(shape1)
    polygons2 = getPolygons(shape2)
    intersections = Polygon()
    for polygon1 in polygons1:
        for polygon2 in polygons2:
            intersections = intersections.union(polygon1.intersection(polygon2))
    return intersections
""" and None


In [20]:
test1 = Polygon(test2000.points)
test2 = Polygon(test2010.points[8:-1])

intersection = test1.intersection(test2)

print(intersection.area)
print(test1.area)
print(test2.area)

3298946.9735208494
4041031.4047255907
3298947.1335533224


In [111]:
getPolygons(test2000).copy()

In [87]:
print(Polygon(test2000.points).area)
print(Polygon(test2010.points).area)
print(findIntersections(test2000, test2010).area)

68327693.3777957
42371938.94567729
38437575.03577941


In [142]:
# for the following dictionary, the key is boroct2000 and value is the corresponding boroct2010
map_2000_to_2010 = {}

for this_ct2000 in unique_2000:
    #potential_pool: e.g. for Manhattan {1XXXXXXX1:polygons_1XXXXXXX1, 1XXXXXXX2:polygons_1XXXXXXX2, ...}
    potential_pool = boroPolygons_pool[str(this_ct2000)[0]]
    max_overlap = 0
    shape00 = sp2000.shape(sp2000_boroct2000_posDict[this_ct2000])
    polygons00 = getPolygons(shape00)
    area00 = Polygon(shape00.points).area
    # for the following dict: {'label':ct2010,'polygons':polygons,'area':area}
    for dict_10 in potential_pool:
        this_intersections = findIntersections(polygons00,dict_10['polygons'])
        intersections_area = this_intersections.area
        this_overlap = max(intersections_area / area00, intersections_area / dict_10['area'])
        if this_overlap > 0.1 and this_overlap > max_overlap:
            max_overlap = this_overlap
            corresponding = dict_10['label']
    if max_overlap > 0:
        map_2000_to_2010[this_ct2000] = corresponding


In [147]:
for this_ct2000 in common:
    map_2000_to_2010[this_ct2000] = this_ct2000
special_ct2000 = unique_2000 - set(map_2000_to_2010.keys())

In [55]:
polygons1 = getPolygons(test2000)
polygons2 = getPolygons(test2010)

In [25]:
print(test2000.parts)
print(test2010.parts)
print(Polygon(test2010.points).area)

[0, 47, 51, 56, 95, 101, 106, 111, 121, 137, 153, 833, 841, 845, 876, 882]
[0, 47, 51, 56, 95, 101, 670, 784, 904, 909, 914, 924, 2171, 3891, 4621, 4748, 5419, 5448, 5547, 5647, 5814, 5918, 5932, 5973, 6175]
42371938.94567729


In [37]:
print('number of ct in both ct2000 and ct2010:')
print(len(common))
print('number of ct only in ct2010')
print(len(unique_2010))
print('number of ct only in ct2000')
print(len(unique_2000))

print(next(iter(common)))

number of ct in both ct2000 and ct2010:
1929
number of ct only in ct2010
237
number of ct only in ct2000
287
3072000


In [149]:
'----------------------------------------------------------------------------------------------------------------------------'

'----------------------------------------------------------------------------------------------------------------------------'

In [156]:
nta2cb = pd.read_excel('nta2cb.xlsx')
cd = pd.read_csv('CommunityDistrict.csv')

borough_code = {'Manhattan':1,'Bronx':2,'Brooklyn':3,'Queens':4,'Staten':5}

def cb2borocd(cb):
    cb_parse = cb.split()
    borough = cb_parse[0]
    num = int(cb_parse[-1])
    return borough_code[borough]*100 + num

nta2cb['BoroCD'] = nta2cb['Community Board'].apply(cb2borocd)

nta2cb.to_csv('nta2cb.csv', index = False)

In [191]:
nta2cb = pd.read_csv('nta2cb.csv')
nta2cb.head(10)

,Community Board,Area km2,Pop. Census 2010,Pop./km2,Neighborhoods,BoroCD
0,Bronx CB 1,7.17,91497,12761,"Melrose, Mott Haven, Port Morris",201
1,Bronx CB 2,5.54,52246,9792,"Hunts Point, Longwood",202
2,Bronx CB 3,4.07,79762,19598,"Claremont, Concourse Village, Crotona Park, Mo...",203
3,Bronx CB 4,5.28,146441,27735,"Concourse, High Bridge",204
4,Bronx CB 5,3.55,128200,36145,"Fordham, Morris Heights, Mount Hope, Universit...",205
5,Bronx CB 6,4.01,83268,20765,"Bathgate, Belmont, East Tremont, West Farms",206
6,Bronx CB 7,4.84,139286,28778,"Bedford Park, Norwood, University Heights",207
7,Bronx CB 8,8.83,101731,11521,"Fieldston, Kingsbridge, Kingsbridge Heights, M...",208
8,Bronx CB 9,12.41,172298,13884,"Bronx River, Bruckner, Castle Hill, Clason Poi...",209
9,Bronx CB 10,16.76,120392,7183,"City Island, Co-op City, Locust Point, Pelham ...",210


In [212]:
BoroCD_nta_dic = {}
for ind in nta2cb.index:
    nta = nta2cb.loc[ind].Neighborhoods.split(',')
    for i in range(len(nta)):
        if nta[i][0] == ' ':
            nta[i] = nta[i][1:]
        if nta[i][-1] == ' ':
            nta[i] = nta[i][:-1]
    BoroCD_nta_dic[nta2cb.loc[ind].BoroCD] = nta

In [213]:
ct2010_nta_dic = {}
for ind in ct2010.index:
    nta = ct2010.loc[ind].ntaname.split('-')
    ct2010_nta_dic[ct2010.loc[ind].boroct2010] = nta

In [215]:
for key,values in ct2010_nta_dic.items():
    for i in values:
        if i not in pool_from_cb:
            print(key,i)

5000900 West New Brighton
5012500 West New Brighton
5013301 West New Brighton
5002002 Ft. Wadsworth
5006400 Ft. Wadsworth
5007400 Ft. Wadsworth
1006800 Gramercy
1008900 Hudson Yards
1008900 Flatiron
1008900 Union Square
1009600 Midtown South
1009800 East Midtown
1010000 East Midtown
1010200 Midtown South
1010400 Midtown South
1011201 Midtown South
1011202 Midtown South
1011203 East Midtown
1011300 Midtown South
1011402 Carnegie Hill
1013000 Carnegie Hill
1014000 Carnegie Hill
1014801 Carnegie Hill
1014802 Carnegie Hill
1015602 East Harlem South
1016001 Carnegie Hill
1018400 East Harlem North
1019000 Central Harlem South
1020102 Central Harlem South
1020600 Central Harlem North
1021800 Central Harlem South
1022000 Central Harlem South
1023400 Central Harlem North
1023900 Washington Heights South
1024900 Washington Heights South
1025300 Washington Heights South
1025700 Central Harlem South
1026100 Washington Heights South
1026300 Washington Heights South
1026700 Washington Heights North


5018100 Lighthouse Hill
5027706 Heartland Village
5027706 Lighthouse Hill
5017700 Heartland Village
5017700 Lighthouse Hill
1003700 Civic Center
1004900 Civic Center
5027705 Heartland Village
5027705 Lighthouse Hill
4095400 Hammels
4094202 Hammels
4094203 Hammels
1009700 Hudson Yards
1009700 Flatiron
1009700 Union Square
1004400 Cooper Village
3007700 Columbia Street
1008400 Midtown South
3002100 Downtown Brooklyn
3054300 park
3054300 cemetery
3054300 etc
3054300 Brooklyn
2033700 North Riverdale
4093401 Broad Channel
4093402 Broad Channel
4093800 Broad Channel
3000200 Sunset Park West
4112300 Bayside Hills
3034800 Seagate
1004100 Civic Center
1016800 East Harlem South
1024100 Washington Heights South
1024700 Washington Heights South
4111300 Bayside Hills
4056100 park
4056100 cemetery
4056100 etc
4056100 Queens
3017700 park
3017700 cemetery
3017700 etc
3017700 Brooklyn
2043500 park
2043500 cemetery
2043500 etc
2043500 Bronx
5005000 Ft. Wadsworth
2011000 Schuylerville
2011000 Edgewater P

In [205]:
pool_from_cb = []
for i in BoroCD_nta_dic.values():
    pool_from_cb.extend(i)
    
pool_from_cb = set(pool_from_cb)

In [208]:
ct2010.loc[3].ntaname.split('-')

['Westerleigh']

In [11]:
print('----------------------------mapping from ct2000 to Community Boarder geometrically--------------------------------------')

----------------------------mapping from ct2000 to Community Boarder geometrically--------------------------------------


In [218]:
ct2000 = pd.read_csv("2000.csv")
ct2000.head()

,CTLabel,BoroCode,BoroName,CT2000,BoroCT2000,CDEligibil,NTACode,NTANAme,PUMA,Shape_Leng,Shape_Area
0,32.0,1,Manhattan,3200,1003200,E,MN22,East Village,3809,6358.386615,2.334190e+06
1,33.0,1,Manhattan,3300,1003300,I,MN24,SoHo-TriBeCa-Civic Center-Little Italy,3810,8170.738948,3.677832e+06
2,34.0,1,Manhattan,3400,1003400,E,MN22,East Village,3809,5503.258021,1.717139e+06
3,38.0,1,Manhattan,3800,1003800,I,MN22,East Village,3809,5841.732079,2.180725e+06
4,39.0,1,Manhattan,3900,1003900,I,MN24,SoHo-TriBeCa-Civic Center-Little Italy,3810,8478.262635,3.000514e+06


In [228]:
import shapefile

sp2000 = shapefile.Reader("nyct2000/nyct2000")

#CB for community board
spCB = shapefile.Reader("nycd_18d/nycd")

print(sp2000.fields)
print(spCB.fields)

#position dictory: record the index of each value of field in the record list (which is the same as the index of correponding shape)
def posDict(sp, field):
    result = {}
    for rec in sp.records():
        result[int(rec[field])] = rec.oid
    return result

sp2000_boroct2000_posDict = posDict(sp2000, 'BoroCT2000')
CB_BoroCD_posDict = posDict(spCB, 'BoroCD')

BoroCD = []
for record in spCB.records():
    BoroCD.append(record.BoroCD)

[('DeletionFlag', 'C', 1, 0), ['CTLabel', 'C', 7, 0], ['BoroCode', 'C', 1, 0], ['BoroName', 'C', 32, 0], ['CT2000', 'C', 6, 0], ['BoroCT2000', 'C', 7, 0], ['CDEligibil', 'C', 1, 0], ['NTACode', 'C', 4, 0], ['NTANAme', 'C', 75, 0], ['PUMA', 'C', 4, 0], ['Shape_Leng', 'F', 19, 11], ['Shape_Area', 'F', 19, 11]]
[('DeletionFlag', 'C', 1, 0), ['BoroCD', 'N', 4, 0], ['Shape_Leng', 'F', 19, 11], ['Shape_Area', 'F', 19, 11]]


In [ ]:
from shapely.geometry import Polygon

def seperatePoints(points, breakings):
    result = []
    for i in range(len(breakings)):
        breaking = breakings[i]
        try:
            next_breaking = breakings[i+1]
        except:
            next_breaking = len(points)
        result.append(points[breaking:next_breaking])
    return result
            

def getPolygons(shape):
    polygons = []
    points = seperatePoints(shape.points, shape.parts)
    for polygon in points:
        polygons.append(Polygon(polygon))
    return polygons

def findIntersections(polygons1,polygons2):
    intersections = Polygon()
    for polygon1 in polygons1:
        for polygon2 in polygons2:
            intersections = intersections.union(polygon1.intersection(polygon2))
    return intersections

In [234]:
# cbPolygons_pool: {'1':{BoroCD11:polygons11, BoroCD12: polygons12, ...},'2':{BoroCD21:polygons21, BoroCD22: polygons22, ...},...}
cbPolygons_pool = {'1':{},'2':{},'3':{},'4':{},'5':{}}
for this_cb in BoroCD:
    this_shape = spCB.shape(CB_BoroCD_posDict[this_cb])
    this_polygons = getPolygons(this_shape)
    cbPolygons_pool[str(this_cb)[0]][this_cb] = this_polygons

In [279]:
# for the following dictionary, the key is boroct2000 and value is the corresponding boroct2010
map_ct2000_to_communityBoard = {}

for this_ct2000 in ct2000.BoroCT2000:
    #potential_pool: e.g. for Manhattan {101:polygons_cb101, 102:polygons_102, ...}
    potential_pool = cbPolygons_pool[str(this_ct2000)[0]]
    max_overlap = 0
    shape00 = sp2000.shape(sp2000_boroct2000_posDict[this_ct2000])
    polygons00 = getPolygons(shape00)
    area00 = 0
    for temp in polygons00:
        area00 = area00 + temp.area
    for boroCD,polygons in potential_pool.items():
        this_intersections = findIntersections(polygons00,polygons)
        intersections_area = this_intersections.area
        this_overlap = intersections_area / area00
        if this_overlap > 0.4 and this_overlap > max_overlap:
            max_overlap = this_overlap
            corresponding = boroCD
    if max_overlap > 0:
        map_ct2000_to_communityBoard[this_ct2000] = corresponding


In [280]:
exception = set(ct2000.BoroCT2000) - set(map_ct2000_to_communityBoard.keys())

for this_ct2000 in exception:
    max_overlap = 0
    shape00 = sp2000.shape(sp2000_boroct2000_posDict[this_ct2000])
    polygons00 = getPolygons(shape00)
    area00 = 0
    for temp in polygons00:
        area00 = area00 + temp.area
    for potential_pool in cbPolygons_pool.values():
        for boroCD,polygons in potential_pool.items():
            this_intersections = findIntersections(polygons00,polygons)
            intersections_area = this_intersections.area
            this_overlap = intersections_area / area00
            if this_overlap > 0.4 and this_overlap > max_overlap:
                max_overlap = this_overlap
                corresponding = boroCD
    if max_overlap > 0:
        map_ct2000_to_communityBoard[this_ct2000] = corresponding


In [281]:
set(ct2000.BoroCT2000) - set(map_ct2000_to_communityBoard.keys())

{1014300,
 2027600,
 2033400,
 2043500,
 2050400,
 3017700,
 3070202,
 3070203,
 4033100,
 4038300,
 4064102,
 4071600,
 4091602,
 4091800,
 4107202,
 5001800,
 5015400}

In [297]:
ct2000_to_CB = pd.DataFrame(list(map_ct2000_to_communityBoard.items()), columns = ['boroCT2000', 'boroCD'])
#ct2000_to_CB.to_csv('ct2000_to_CB.csv', index = False)

In [12]:
print('--------------------------------------------------finished------------------------------------------------------------')

--------------------------------------------------finished------------------------------------------------------------


In [17]:
print('-------------------------------locate netlogo patches with real world district---------------------------------------------')

-------------------------------locate netlogo patches with real world district---------------------------------------------


In [157]:
import numpy as np
import pandas as pd
import shapefile

#CB for community board
spCB = shapefile.Reader("netlogo model/nycd/nycd")

In [158]:
def bbox_netlogo(*arg):
    if len(arg) == 1:
        return [-arg[0], -arg[0], arg[0], arg[0]]
    if len(arg) == 2:
        return [-arg[0], -arg[1], arg[0], arg[1]]


# transform from coord in world1 (specified by bbox1) to coord in world2 x2 = ax * x1 + bx, y2 = ay * y1 + by
# return [ax,ay,bx,by]
def transform_world(bbox1, bbox2):
    xrange1 = bbox1[2] - bbox1[0]
    xrange2 = bbox2[2] - bbox2[0]
    yrange1 = bbox1[3] - bbox1[1]
    yrange2 = bbox2[3] - bbox2[1]
    ax = xrange2 / xrange1
    ay = yrange2 / yrange2
    bx = (bbox2[2] + bbox2[0]) - ax * (bbox1[2] + bbox1[0])
    by = (bbox2[3] + bbox2[1]) - ay * (bbox1[3] + bbox1[1])
    return ax, ay, bx, by

# above two functions are not actually used here
# fit really world coord to netlogo coord, x' = a * x + bx, y' = a * y + by where x' and y' are corresponding coord of really world coord x and y in netlogo world
def fit_netlogo_world(bbox1, *nlcor):
    assert len(nlcor) < 3, "wrong input for netlogo world"
    if len(nlcor) == 1:
        xnl = nlcor[0]
        ynl = nlcor[0]
    if len(nlcor) == 2:
        xnl = nlcor[0]
        ynl = nlcor[1]
    xrange1 = bbox1[2] - bbox1[0]
    xrange_nl = 2 * xnl + 1
    yrange1 = bbox1[3] - bbox1[1]
    yrange_nl = 2 * ynl + 1
    if xrange1 * yrange_nl >= xrange_nl * yrange1:
        a = xrange_nl / xrange1
        bx = -a * (bbox1[2] + bbox1[0]) / 2
        by = -a * (bbox1[3] + bbox1[1]) / 2
    else:
        a = yrange_nl / yrange1
        bx = -a * (bbox1[2] + bbox1[0]) / 2
        by = -a * (bbox1[3] + bbox1[1]) / 2
    return a, bx, by

In [159]:
from shapely.geometry import Polygon

def seperatePoints(points, breakings):
    result = []
    for i in range(len(breakings)):
        breaking = breakings[i]
        try:
            next_breaking = breakings[i+1]
        except:
            next_breaking = len(points)
        result.append(points[breaking:next_breaking])
    return result

def getPolygons(shape):
    polygons = []
    points = seperatePoints(shape.points, shape.parts)
    for polygon in points:
        polygons.append(Polygon(polygon))
    return polygons

def transform_points(points, a, bx, by):
    result = []
    for points_set in points:
        result.append(np.array(points_set) * [a,a] + [bx, by])
    return result

def getTransformedPolygons(shape, a, bx, by):
    polygons = []
    points = seperatePoints(shape.points, shape.parts)
    points = transform_points(points, a, bx, by)
    for polygon in points:
        polygons.append(Polygon(polygon))
    return polygons

In [160]:
a, bx, by = fit_netlogo_world(spCB.bbox, 50)
patches_mapping = {}

def half_round(x):
    result =  int(x + 0.5)
    boundary = 50
    if abs(result) > boundary:
        return np.sign(result) * boundary
    else:
        return result

def intersection_by_point(x,y,polygon, len_side = 1):
    diff = len_side / 2
    polygon_xy = Polygon([(x - diff, y + diff), (x + diff, y + diff), (x + diff, y - diff), (x - diff, y - diff)])
    return polygon_xy.intersection(polygon)

patches_mapping = {}

for x in np.arange(-50, 50 + 1):
    for y in np.arange(-50, 50 + 1):
        patches_mapping[(x, y)] = [0,0]
        
for i in range(len(spCB)):
    this_patches_mapping = {}
    this_shape = spCB.shape(i)
    this_boroCD = spCB.record(i).BoroCD
    this_polygons = getTransformedPolygons(this_shape, a, bx, by)
    for polygon in this_polygons:
        xmin, ymin, xmax, ymax = polygon.bounds
        for x in np.arange(half_round(xmin), half_round(xmax)):
            for y in np.arange(half_round(ymin), half_round(ymax)):
                try:
                    this_patches_mapping[(x,y)] = this_patches_mapping[(x,y)].union(intersection_by_point(x,y,polygon))
                except:
                    this_patches_mapping[(x,y)] = Polygon()
                    this_patches_mapping[(x,y)] = this_patches_mapping[(x,y)].union(intersection_by_point(x,y,polygon))
    for cord, intersec in this_patches_mapping.items():
        this_patches_mapping[cord] = intersec.area / 1
        if this_patches_mapping[cord] > patches_mapping[cord][1]:
            patches_mapping[cord][0] = this_boroCD
            patches_mapping[cord][1] = this_patches_mapping[cord]

In [161]:
patches_boroCD = pd.DataFrame(columns = ['x','y','boroCD'])

for cord, mapping in patches_mapping.items():
    #mapping: [boroCD, percentage of overlapping]
    if mapping[1] > 0.4:
        patches_boroCD = patches_boroCD.append({'x':cord[0],'y':cord[1],'boroCD':mapping[0]}, ignore_index = True)

In [163]:
patches_boroCD.head()

,x,y,boroCD
0,-50,-37,503
1,-50,-36,503
2,-49,-37,503
3,-49,-36,503
4,-49,-35,503


In [164]:
patches_boroCD.to_csv('patches_boroCD.csv',index = False)

In [137]:
for polygon in this_polygons:
    xmin, ymin, xmax, ymax = polygon.bounds
    for x in np.arange(half_round(xmin), half_round(xmax)):
        for y in np.arange(half_round(ymin), half_round(ymax)):
            try:
                this_patches_mapping[(x,y)] = this_patches_mapping[(x,y)].union(intersection_by_point(x,y,polygon))
            except:
                this_patches_mapping[(x,y)] = Polygon()
                this_patches_mapping[(x,y)] = this_patches_mapping[(x,y)].union(intersection_by_point(x,y,polygon))
for cord, intersec in this_patches_mapping.items():
    this_patches_mapping[cord] = intersec.area / 1
    if this_patches_mapping[cord] > patches_mapping[cord][1]:
        patches_mapping[cord][0] = this_boroCD
        patches_mapping[cord][1] = this_patches_mapping[cord]

In [139]:
this_patches_mapping

{(-6, -33): 0.0,
 (-6, -32): 0.013354027271316914,
 (-6, -31): 0.6824934733292242,
 (-6, -30): 0.4052999680527697,
 (-6, -29): 0.0,
 (-6, -28): 0.0,
 (-6, -27): 0.0,
 (-5, -33): 0.0,
 (-5, -32): 0.21747525717099225,
 (-5, -31): 1.0,
 (-5, -30): 0.8647361413796694,
 (-5, -29): 0.0,
 (-5, -28): 0.0,
 (-5, -27): 0.0,
 (-4, -33): 0.0008559788007483294,
 (-4, -32): 0.8034140973545765,
 (-4, -31): 1.0,
 (-4, -30): 0.7666381558639028,
 (-4, -29): 0.3506320344670205,
 (-4, -28): 0.003540354657830098,
 (-4, -27): 0.0,
 (-3, -33): 0.0,
 (-3, -32): 0.8575653754230375,
 (-3, -31): 1.0,
 (-3, -30): 0.4289074277159796,
 (-3, -29): 0.8980819474975806,
 (-3, -28): 0.449220097973723,
 (-3, -27): 0.0,
 (-2, -33): 0.0,
 (-2, -32): 0.7514309299968706,
 (-2, -31): 1.0,
 (-2, -30): 0.6655074255913633,
 (-2, -29): 0.9036094417285874,
 (-2, -28): 0.9873413822430485,
 (-2, -27): 0.4329721271360764,
 (-1, -33): 0.0009780901441513406,
 (-1, -32): 0.6256288681250919,
 (-1, -31): 1.0,
 (-1, -30): 0.981311244995214

In [130]:
for i in range(len(spCB.bbox)):
    if i % 2 == 0:
        print(spCB.bbox[i] * a + bx)
    else:
        print(spCB.bbox[i] * a + by)

-50.5
-50.01369483974162
50.5
50.01369483974156


313

In [111]:
for i in range(len(spCB)):
    this_patches_mapping = {}
    this_shape = spCB.shape(i)
    this_boroCD = spCB.record(i).BoroCD
    this_polygons = getPolygons(this_shape)
    for polygon in this_polygons:
        xmin, ymin, xmax, ymax = polygon.bounds
        for x in np.arange(half_round(xmin), half_round(xmax)):
            for y in np.arange(half_round(ymin), half_round(ymax)):
                try:
                    this_patches_mapping[(x,y)] = this_patches_mapping[(x,y)].union(intersection_by_point(x,y,polygon))
                except:
                    this_patches_mapping[(x,y)] = Polygon()
                    this_patches_mapping[(x,y)] = this_patches_mapping[(x,y)].union(intersection_by_point(x,y,polygon))
    for cord, intersec in this_patches_mapping.items():
        this_patches_mapping[cord] = intersec.area / 1
        if this_patches_mapping[cord] > patches_mapping[cord][1]:
            patches_mapping[cord][0] = this_boroCD
            patches_mapping[cord][1] = this_patches_mapping[cord]

KeyboardInterrupt: 

In [103]:
x = Polygon([(0,0),(1,1),(2,0),(1,-1)])
x2 = [Polygon([(0,0.5),(0.5,0.5),(0.5,0),(0,0)]), Polygon([(0.6,0.2),(0.8,0.2),(0.8,0),(0.6,0)])]

In [104]:
x.intersection(x2)

AttributeError: 'list' object has no attribute '_geom'

In [99]:
x.bounds

(0.0, -1.0, 2.0, 1.0)

In [49]:
m = np.array([[1],[1]])

In [79]:
t = []
t.append([1,2,3])

In [80]:
print(t)

[[1, 2, 3]]


In [132]:
z = x.copy()

In [133]:
x['b'] = 4

In [134]:
z

{'a': 3, 'b': [{3: [3, 3]}, {4: [4, 4]}]}

In [107]:
x = (1,2)

In [108]:
x[0] = 3

TypeError: 'tuple' object does not support item assignment

In [105]:
x = {}
x[(1,2)] = 6

In [106]:
x

{(1, 2): 6}

In [140]:
0.0 > 0

False

In [173]:
x = pd.DataFrame({'a':[1,2,3],'b':[4,5,6]})

In [174]:
x

,a,b
0,1,4
1,2,5
2,3,6


In [175]:
for i in x:
    print(i)

a
b


In [181]:
x = 'adasdas'

In [182]:
y = x

In [184]:
x = x[3:]

In [33]:
int(6.9)

6

In [292]:
pd.DataFrame([(1,2),(3,4),(5,6)])

,0,1
0,1,2
1,3,4
2,5,6


In [202]:
x

{1, 2, 3}

In [204]:
x.add(4)

In [59]:
np.dot([(1,2),(3,4),(5,6)],[[1],[2]])

array([[ 5],
       [11],
       [17]])